# Secure your Azure SQL Database
Secure your Azure SQL Database to keep your data secure and diagnose potential security concerns as they happen.

In this module, you will:

1. Control network access to your Azure SQL Database using firewall rules
2. Control user access to your Azure SQL Database using authentication and authorization
3. Protect your data in transit and at rest
4. Audit and monitor your Azure SQL Database for access violations

## Exercise - Secure a database
First, to show how to secure your database, set up the following resources
- A Linux VM named appServer. This server will act as the application server that users would be connecting to, and will need to connect to the database. We'll install sqlcmd on the VM to simulate an application running on appServer making connections to the database.
- An Azure SQL Database logical server. This logical server is needed to host one or more databases.
- A database on our logical server called marketplaceDb. We'll create it using the AdventureWorksLT demo database so we have some tables and data to work with. This will include some sensitive data, such as email addresses and phone numbers that we'll want to make sure are properly secured.

### Create Azure SQL DB

Step 1: In Bash set up variables 

Set an admin login and password for your database

`export ADMINLOGIN='[ServerAdmin]'
export PASSWORD='[password]'`

Set the logical SQL server name. We'll add a random string as it needs to be globally unique.

`export SERVERNAME=server$RANDOM
export RESOURCEGROUP=learn-070869d0-8594-4673-8283-02d536c2e17d`

Set the location, we'll pull the location from our resource group. REMEMBER THIS

`export LOCATION=$(az group show --name $RESOURCEGROUP | jq -r '.location')`

Step 2: Create Azure SQL DB logical server

`az sql server create \
    --name $SERVERNAME \
    --resource-group $RESOURCEGROUP \
    --location $LOCATION \
    --admin-user $ADMINLOGIN \
    --admin-password "$PASSWORD"`
   

Step 3: Create the `marketplaceDb` on the logical server just created, use *AdventureWorksLT* databse as template

`az sql db create --resource-group $RESOURCEGROUP \
    --server $SERVERNAME \
    --name marketplaceDb \
    --sample-name AdventureWorksLT \
    --service-objective Basic`

Step 4: Get connection string for DB

`az sql db show-connection-string --client sqlcmd --name marketplaceDb --server $SERVERNAME | jq -r`

Note: I don't understand the | jq -r, I know | is piping to JSON parsing, however the only difference in output is that this bit removes quotations on same output

### Create and configure a Linux virtual machine
Step 1: Create VM

`az vm create \
  --resource-group $RESOURCEGROUP \
  --name appServer \
  --image UbuntuLTS \
  --size Standard_DS2_v2 \
  --generate-ssh-keys`
  

Step 2: Connect to public IP address (output of the VM creation)

`ssh nnn.nnn.nnn.nn`

Step 3: install mssql-tools on Linux VM to connect to database through `sqlcmd`

`echo 'export PATH="$PATH:/opt/mssql-tools/bin"' >> ~/.bash_profile
echo 'export PATH="$PATH:/opt/mssql-tools/bin"' >> ~/.bashrc
source ~/.bashrc
curl https://packages.microsoft.com/keys/microsoft.asc | sudo apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list | sudo tee /etc/apt/sources.list.d/msprod.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get install -y mssql-tools unixodbc-dev`

I tried typing this all in myself, nothing here worked because I forgot to SSH. Second time I just pasted it all in

## Ways to control access to SQL Database
1. Firewall rules- initially all public access to DB is blocked by the SQL Db firewall. Rules are configured at teh server and/or database level. Depending on the level, rules you can apply will be as follows:
  - Server-level Firewall rules - can all be created and manipulated through portal, PowerShell, CLI, and through Transact-SQL (T-SQL)
    - Allow Access to Azure Services
    
      <img src="https://docs.microsoft.com/en-us/learn/modules/secure-your-azure-sql-database/media/2-allow-azure-services.png">
    - IP address rules
    
      <img src="https://docs.microsoft.com/en-us/learn/modules/secure-your-azure-sql-database/media/2-server-ip-rule-1.png">
    - Virtual network rules - limit access to subnets, VNet Address spaces private so you can eliminate exposure to public IP addresses
    
      <img src="https://docs.microsoft.com/en-us/learn/modules/secure-your-azure-sql-database/media/2-vnet-rule.png">
  - Database-level firewall rules - When replicating a database to another server, the database-level rules will be replicated, since they are stored in the database itself. Downside is only IP address rules. Can only be created and manipulated through T-SQL
    - IP address rules
    
      <img src="https://docs.microsoft.com/en-us/learn/modules/secure-your-azure-sql-database/media/2-db-ip-rule-1.png">